# Домашнее задание № 10 (демо)<br>Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('../Data/flight_delays_train.csv')
test = pd.read_csv('../Data/flight_delays_test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [ ]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [ ]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

In [ ]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

# pd.Series(logit_test_pred, 
#           name='dep_delayed_15min').to_csv('../Data/logit_2feat.csv', 
#                                            index_label='id', 
#                                            header=True)

Как был получен бенчмарк в соревновании:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, мы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

<hr>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train = train.copy()
df_train['route'] = df_train['Origin'] + '-' + df_train['Dest']
y_df_train = df_train['dep_delayed_15min'].map({'Y': 1, 'N': 0})
df_train.drop(['Origin', 'Dest', 'dep_delayed_15min'], axis=1, inplace=True)

Нет пустых значений. Все заполнены

In [ ]:
def df_preparation(df_train, df_test):
    common = df_train.merge(df_test, how='outer', indicator=('where_is')) # объединение фреймов
    common['route'] = common['Origin'] + '-' + common['Dest']
    y = common['dep_delayed_15min']
    OHE = pd.get_dummies(common.drop(['dep_delayed_15min'], axis=1))
    return OHE

    
    
df_preparation(train, test)

MemoryError: Unable to allocate 963. MiB for an array with shape (5048, 200000) and data type uint8

In [ ]:
cats = df_train.loc[:, df_train.dtypes=='object'].columns # категориальные фичи
df_train[cats]

,Month,DayofMonth,DayOfWeek,UniqueCarrier,route
0,c-8,c-21,c-7,AA,ATL-DFW
1,c-4,c-20,c-3,US,PIT-MCO
2,c-9,c-2,c-5,XE,RDU-CLE
3,c-11,c-25,c-6,OO,DEN-MEM
4,c-10,c-7,c-6,WN,MDW-OMA
...,...,...,...,...,...
99995,c-5,c-4,c-3,OO,SFO-RDD
99996,c-1,c-18,c-3,CO,EWR-DAB
99997,c-1,c-24,c-2,NW,DTW-IAH
99998,c-4,c-27,c-4,MQ,DFW-GGG


In [ ]:
df_train = pd.get_dummies(df_train, prefix_sep=': ')

In [ ]:
df_test = test.copy()
df_test

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258
...,...,...,...,...,...,...,...,...
99995,c-6,c-5,c-2,852,WN,CRP,HOU,187
99996,c-11,c-24,c-6,1446,UA,ORD,LAS,1515
99997,c-1,c-30,c-2,1509,OO,ORD,SGF,438
99998,c-1,c-5,c-5,804,DL,LGA,ATL,761


In [ ]:
df_test['route'] = df_test['Origin'] + '-' + df_test['Dest']
df_test.drop(['Origin', 'Dest'], axis=1, inplace=True)

In [ ]:
df_test = pd.get_dummies(df_test, prefix_sep=': ')

In [ ]:
df_train.shape, df_test.shape

((100000, 4503), (100000, 4741))

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df_train, 
                                                      y_df_train, 
                                                      test_size=0.3, 
                                                      random_state=2)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict

In [ ]:
df_test

,DepTime,Distance,Month: c-1,Month: c-10,Month: c-11,Month: c-12,Month: c-2,Month: c-3,Month: c-4,Month: c-5,...,route: XNA-MSP,route: XNA-ORD,route: XNA-SLC,route: YAK-CDV,route: YAK-JNU,route: YKM-SLC,route: YUM-IPL,route: YUM-LAS,route: YUM-LAX,route: YUM-PHX
0,615,598,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,739,1235,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,651,577,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1614,377,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1505,258,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,852,187,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,1446,1515,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,1509,438,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99998,804,761,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
cat_params = {}

In [ ]:
cat_model = CatBoostClassifier(iterations=10, 
                               depth=2, 
                               learning_rate=1, 
                               loss_function='Logloss', 
                               verbose=True)

In [ ]:
cat_model.fit(df_train, y_train)

CatBoostError: Length of label=70000 and length of data=100000 is different.

In [ ]:
cat_model.predict()